In [632]:
## Web craping to JSON file

In [633]:
import requests
import sqlalchemy
import getpass
import pandas as pd
from bs4 import BeautifulSoup

In [634]:
# - Define target URL and file path for future use
# - Create dataframe for future use

url = 'https://fastestlaps.com/lists/top-fastest-production-cars'
json_path = r'C:\Users\vaino\Desktop\Python_jupyter\Fastest_Cars.json'
df = pd.DataFrame(columns=["Rank", "Make and model", "Top speed"])

In [635]:
# - Crate variables to hold credentials for the database connection
# - HINT! password is 12345 :)

table_name = 'fastest_cars'
db_user = 'postgres'
db_password = getpass.getpass("Enter Password")
db_host = 'localhost'
db_port = '5432'
db_name = 'postgres'

Enter Password ········


In [636]:
# - Load essentials from webpage

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', class_ = 'table table-striped')
rows = table.find_all('tr')[1:]

In [637]:
# - Extract the data
# - Write the data in the dataframe created earlier

for row in rows:
        col = row.find_all('td')
        if len(col) != 0:
            data_dict = {"Rank": col[0].get_text(strip=True),
                         "Make and model": col[1].get_text(strip=True),
                         "Top speed": col[2].get_text(strip=True)}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df, df1], ignore_index=True)
        else:
            break

In [638]:
# - Cheeck if the scraping was successful

df

,Rank,Make and model,Top speed
0,1.,Koenigsegg Agera RS (1MW),447 kph (278 mph)
1,2.,Koenigsegg Agera R,443 kph (275 mph)
2,3.,Bugatti Chiron Super Sport,440 kph (273 mph)
3,4.,9ff GT9 Vmax,437 kph (272 mph)
4,5.,Koenigsegg One:1,437 kph (272 mph)
...,...,...,...
95,96.,Nissan R390 GT1,355 kph (221 mph)
96,97.,Aston Martin One-77,355 kph (221 mph)
97,98.,Bugatti EB110 SS,355 kph (221 mph)
98,99.,McLaren F1 GT,355 kph (221 mph)


In [639]:
df['Top speed'] = df['Top speed'].str.split().str[:2]


In [640]:
df

,Rank,Make and model,Top speed
0,1.,Koenigsegg Agera RS (1MW),"[447, kph]"
1,2.,Koenigsegg Agera R,"[443, kph]"
2,3.,Bugatti Chiron Super Sport,"[440, kph]"
3,4.,9ff GT9 Vmax,"[437, kph]"
4,5.,Koenigsegg One:1,"[437, kph]"
...,...,...,...
95,96.,Nissan R390 GT1,"[355, kph]"
96,97.,Aston Martin One-77,"[355, kph]"
97,98.,Bugatti EB110 SS,"[355, kph]"
98,99.,McLaren F1 GT,"[355, kph]"


In [641]:
# - Check data types

df.dtypes

Rank              object
Make and model    object
Top speed         object
dtype: object

In [642]:
# - Use lambda function to format the values in the Top speed column

df['Top speed'] = df['Top speed'].apply(lambda x: [x[0].replace('[', ''), x[1].replace(']', '').replace('kph', 'km/h')])
df['Top speed'] = df['Top speed'].apply(lambda x: ' '.join(x))

In [643]:
df

,Rank,Make and model,Top speed
0,1.,Koenigsegg Agera RS (1MW),447 km/h
1,2.,Koenigsegg Agera R,443 km/h
2,3.,Bugatti Chiron Super Sport,440 km/h
3,4.,9ff GT9 Vmax,437 km/h
4,5.,Koenigsegg One:1,437 km/h
...,...,...,...
95,96.,Nissan R390 GT1,355 km/h
96,97.,Aston Martin One-77,355 km/h
97,98.,Bugatti EB110 SS,355 km/h
98,99.,McLaren F1 GT,355 km/h


In [644]:
# - Split column 'Make and model' into two columns 'Make' and 'Model'
# - this makes future grouping easier

df[['Make','Model']] = df['Make and model'].str.split(' ', n=1, expand=True)
df = df.drop('Make and model', axis=1)

In [645]:
df['Make'].value_counts()

Make
Koenigsegg        12
9ff               10
Bugatti           10
RUF                8
Brabus             6
McLaren            4
Ultima             3
Ferrari            3
Porsche            3
Lamborghini        2
Pagani             2
Toyota             2
Zenvo              2
Lotec              2
Hennessey          2
Dauer              2
SSC                2
Orca               1
Jaguar             1
Aston              1
Nissan             1
Bristol            1
Pininfarina        1
Renault-Alpine     1
Gumpert            1
Czinger            1
Fahlke             1
Noble              1
B                  1
Isdera             1
De                 1
Aspark             1
Leblanc            1
Rimac              1
Rossin-Bertin      1
Mercedes-Benz      1
WM                 1
Koenig             1
HTT                1
Saleen             1
W                  1
Arash              1
Name: count, dtype: int64

In [646]:
# - Check for errors or inconveniences and correct if needed
# - Lets check the following values for mistakes: Aston, B, De and W
# - Here we will use the query method, although not necessary

df.query("Make == 'Aston' or Make == 'B' or Make == 'De' or Make == 'W'")

,Rank,Top speed,Make,Model
37,38.,390 km/h,W,Motors LykanHypersport
66,67.,370 km/h,De,Macross Epique GT1
71,72.,365 km/h,B,Engineering Edonis
96,97.,355 km/h,Aston,Martin One-77


In [647]:
# - I searched the car models from the web and corrected them below

df.loc[df['Make'] == 'W', 'Make'] = 'W Motors'
df.loc[df['Make'] == 'De', 'Make'] = 'De Macross'
df.loc[df['Make'] == 'B', 'Make'] = 'B Engineering'
df.loc[df['Make'] == 'Aston', 'Make'] = 'Aston Martin'

# - Remove the first word from the 'Model' column do it doesn't present twice
# - 'Aston Martin' 'Martin One-77' -> 'Aston Martin' 'One-77'

df.loc[df['Model'] == 'Motors LykanHypersport', 'Model'] = 'LykanHypersport'
df.loc[df['Model'] == 'Macross Epique GT1', 'Model'] = 'Epique GT1'
df.loc[df['Model'] == 'Engineering Edonis', 'Model'] = 'Edonis'
df.loc[df['Model'] == 'Martin One-77', 'Model'] = 'One-77'

In [648]:
# - Create engine for database connection
# - Use the credentials created earlier and remember to dispose the engine

credentials = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = sqlalchemy.create_engine(credentials)

df.to_sql(table_name, engine, if_exists='replace', index=False)

engine.dispose()

In [649]:
df.to_json(json_path, orient='records')